<a href="https://colab.research.google.com/github/deepakagrawal/Pandas_Tutorial/blob/master/Data_Wrangling_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Clinic

The objective of python clinic is to 
- create a platform for all Python enthusiasts  
- provide learning resources for all members via this channel and offline. 

Join the slack channel [#topic-python-ds-ml](https://americanairlines.slack.com/messages/CJ7JNTUR4/)

# Python for Data Wrangling

We will use Titanic Survival prediction dataset as an example. We will focus on the data preparation step for this model.

Today we will cover following topics -

* Reading data files into Python
* What is a DataFrame?
* Accessing Main componenets of DataFrame
* How to handle `Missing values`?
* How to add/remove columns/rows?
* How to handle large datasets?
* How to perform groupby/aggregation?
* How to apply a function to columns/rows?
* How to merge/join datasets?


## Problem

Titanic dataset provides information on the fate of passengers on the Titanic, summarized according to economic status (class), sex, age and survival. 

Let's start by importing some of the libraries we know we will need. 

* `pandas` -  This is the most important library for data wrangling in python
* `numpy` - Provides helpful mathematical functions
* `seaborn` - for creating plots and visualizing trends in data

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
file_path='https://raw.githubusercontent.com/deepakagrawal/Pandas_Tutorial/master/data/titanic.csv'

### Use pandas to read the data into the dataframe

In [0]:
df = pd.read_csv(file_path)

## For csv file
# df = pd.read_csv("file.csv")

#### Reading large data

`csv` file format is not ideal data format when handeling data of `>1GB`. In that case file formats such as `parquet` or `hdf5` are more suitable because read/write speeds are higher. Pandas supports all these formats.

If `csv` file formar is unavoidable reading data in chunk can help.

In [0]:
df1 = pd.read_csv(file_path, chunksize=500)
for chunk in df1:
  ## perform operation
  print(chunk.shape)

In [0]:
df1 = pd.read_csv(file_path, memory_map=True)
print(df.info())

#### Reading from Mosaic

In [0]:
## pyodbc is an open source library which makes accessing ODBC databases simple
# import pyodbc

## getpass gets user name and password
# import getpass
# user_id=getpass.getuser()
# print("connecting user : "+ user_id)
# ###################################################
# connection = pyodbc.connect('DSN=MOSAIC_PROD;Uid=' + user_id + ';Pwd=' + getpass.getpass()) ## this assumes that in teradata your connection is saved as "MOSAIC_PROD"
# dat = pd.read_sql("select top 10 * from PROD_REFERENCE_DATA_VWS.AIRPORT_STATION",con=connection)

# print(dat)

# connection.close()

For more details please visit [pandas io guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)

For connecting to mosaic from ORION server please visit [here](https://ghe.aa.com/00740677/ORION-python_pkgs)

### Data Frame properties
Let's take a peek at the dataset and see what does the data look like.

In [0]:
df.shape ## Gives us size of the dataset. No. of rows and columns

In [0]:
df.head()

We see that each row of this dataset represent a single passenger. The definition of the columns is given below

| Column name | Definition |
|------------------------|-----------------|
|pclass | Class of Travel (1=1st; 2=2nd; 3=3rd)|
|survived | Survived or Not (0= No; 1=Yes)|
|name | Passenger Name |
|sex | Genger of the passenger|
| age | Age of the passenger|
| sibsb| Number of Sibling/Spouse aboard|
| parch | Number of Parent/Child aboard|
|fare | The fare which passenger paid for his/her ticket (British Pound)|
|cabin| The cabin which the passenger booked|
|embarked| The port in which a passenger has embarked. C - Cherbourg, S - Southampton, Q = Queenstown|
|boat| Lifeboat (if survived) |
|body | Body number (if did not survive and body was recovered)|
|home.dest|Home/Destination|



Our task is to predict whether the passenger survived or not. So the target column is **`Survived`**. Before we even start building the model we need to do some exploratory analysis to see which features might contribute to a passenger surviving or not.

Let's look at the data types of these columns. We use `DataFrame.info` method to print a concise summary of a DataFrame. We can print the column types, number of missing values and memory usage with this command

In [0]:
df.info(memory_usage=True) # This prints a concise summary of a DataFrame.

Above results shows us that 

1. Total 1309 rows and 14 columns in the DataFrame.
2. There are 4 columns with int64 entries, 3 columns with float64 and 7 columns are string or other objects
3. The total memory used by this dataframe is ~143 KB

Now let's look at the summary of the dataset to see some simple statistics

In [0]:
df.describe(include='all')

From above results we can see that 

1. `pclass` column has minimum 1 and maximum 3.
2. `survived` column has only minimum 0 and maximum 1. 
3. There are 1307 unique names. And we see that freq = 2. That means two passengers have multiple records
4. There are two values for `sex`. And majority are males (843) rest should be female (466)
5. Youngest person Aboard is 0.16 years old and oldest person is 80 years old.
6. The minimum number of Siblings/Sposes aboard are 0 and maximum are 8
7. The minimum number of Parents/Children aboard are 0 and maximum are 9
8. Several passengers were on the same ticket
9. The minimum Fare paid was 0 and maximum paid was 512.33
10. There are 186 unique cabins
11. There are 3 ports the passengers departed from. Contains 2 missing values
12. Total of 486 boats were used
13. Total of 182 bodies were recovered
14. Total of 745 destinations were registered

Let's look at what's happening with the duplicate names.

In [0]:
df[df.name.isin(df.name[df.name.duplicated()])]

Since they paid different fare and their ages are different. We can assume that these are different passengers

### Reducing Memory usage

Now let's see the memory usage for individual column. 

**Note:** This is useful if you have a large dataset and not enough RAM to handle it.

In [0]:
df.memory_usage(index=False)

We can see that all of the columns are using ~10 KB memory. Let's see if we can try to reduce some memory by changing data types of some columns. From the summary results we saw that `Survived` had binary values. Let's see if we convert them to binary how much memory can we save

In [0]:
df.survived.astype('int8').memory_usage()

We can see that by converting the `Survived` column to `int8` from `int64` we reduced the memory usage to 1389 B only. Similarly we can convert `pclass`,  `sibsp` and `parch` columns to `int8`. 

We can also convert `age` and `fare` column to `float32` if we don't need age with more than 3 digits after decimal. Instead of making these changes after read the whole file. We can make these changes at the time of reading the csv file.

In [0]:
df = pd.read_csv(file_path, 
                 dtype={'survived': 'int8', 'pclass': 'int8', 'age':'float32', 'sibsp': 'int8', 'parch': 'int8', 'fare':'float32', 'sex': 'category', 'pclass': 'category', 'embarked': 'category'})

In [0]:
df.memory_usage(index=False)

In [0]:
df.info()

We can see that we have reduced the memory used to 79.5 KB

We see that there are several columns with missing values. e.g. age, fare, cabin, embarked, boat, body and home.dest

### Creating new columns and deleting columns

Let's create a column which has the title of the passenger. We can create this column by separating it from the `Name` column. We see that Title is always the first value

In [0]:
df.name.head()

In [0]:
df['title'] = df.name.apply(lambda x: x.split(',')[1].split('.')[0].strip()) # str.stip() removes all leading and trailing whitespaces
df.head()

Now we can remove the name coulmn since we do not need it.

In [0]:
df.drop(columns='name', inplace=True)
print(df.columns.tolist())

### Creating summary tables (Groupby)

Let's find the number of passengers based on their Titles and gender

In [0]:
df.groupby(['sex', 'title']).size()

We can reassign `Mlle`, `Ms` as `Miss` and `Mme` as `Mrs`.

In [0]:
df.loc[df.title.isin(['Mlle', 'Ms']), 'title'] = 'Miss'
df.loc[df.title == 'Mme', 'title'] = 'Mrs'
df.groupby(['sex', 'title']).size()

Let's look at by title what is the likelihood of surviving

In [0]:
df[["survived", "title"]].groupby("title").mean().plot(kind='bar')

We see that passengers with title `Mr` had the lowest survival rate. On the other hand Miss and Mrs had higher survival rate.

Now let's look at survival rate by number of family members.

In [0]:
df["total_members"] = df.sibsp + df.parch + 1
df[["survived", "total_members"]].groupby('total_members').mean().plot(kind='bar')

We can see passengers with family size more than 4 or single passengers had a low survival rate compared to others.


### Missing values

Let's look at the total number of missing values by each column and see how can we handle them

In [0]:
df.isna().sum(axis=0)

Let's remove unnecessary columns. 

In [0]:
df.drop(columns=['cabin', 'boat', 'body', 'home.dest', 'ticket'], inplace=True)

In [0]:
df.columns.tolist()

Now let's look at the missing values. 

In [0]:
df[df.fare.isnull()]

Let's try to imupte the missing fare. Let's see which columns are correlated with fare. 

In [0]:
fare_by_class_embarked = df.groupby(['pclass', 'embarked'])['fare'].mean().reset_index()
sns.scatterplot(x='pclass', hue='embarked', y='fare', data=df)
print(fare_by_class_embarked)

We see that for passengers of `pclass`=`3` who embarked from `S` the average fare was 14.43, so we can use that fare to impute the missing fare value.

In [0]:
df.loc[(df.fare.isnull()) & (df.pclass==3) & (df.embarked=="S"), "fare"] = df.loc[(df.pclass==3) & (df.embarked=="S"), "fare"].mean()

In [0]:
df[df.embarked.isna()]

Let's look at the relationship of `embarked`, `fare` and `survived`

In [0]:
df[['embarked', 'survived', 'fare']].groupby(['embarked']).mean()

We can see that passengers who embarked from `C` had the highest survival rate and highest fare. Since, the two passengers with missing value for `embarked` have fare of 80 £ and survived so we can impute the missing value as `C`

In [0]:
df.loc[(df.embarked.isna()), "embarked"] = 'C'

Now let's take care of the missing values in `age`. Let's see age distribution first

In [0]:
df.age.plot(kind='hist', edgecolor='black')

### Using seaborn to create visualizations

Let's see similar plot created using `seaborn` library

In [0]:
sns.distplot(df.age[~df.age.isna()])

Age can be predicted based of `title` and `sex` of existing customers and imputing the median age value. Let's first create a scatter plot to see the relationship

In [0]:
labels = df.title.unique()
p = sns.scatterplot(x="title", y="age", hue="sex", data=df)
p.set_xticklabels(labels, rotation=45, horizontalalignment='right')

We see that there is a clear relationship between `age` and the combination of `sex`, `title`. Let's see this relationship using a `pivot table`.

In [0]:
age_pivot_table = df.pivot_table(index=['sex', 'title'], values=['age'], aggfunc='median')
print(age_pivot_table)

### Merging Dataframes

Now there are 2 ways we can impute the missing values in the `age` column - 
1. Write a long if else loop to impte the values.
2. Use `merge` operation to do it more effectively

In [0]:
df_age_imputed = df[df.age.isna()].drop(columns='age').merge(age_pivot_table.reset_index(), on=['sex', 'title'], how='left')
print(df_age_imputed.shape)
df_age_imputed.head()


In [0]:
df = pd.concat([df[~df.age.isna()], df_age_imputed], ignore_index=True, sort=False)

Now let's look at the `age` distribution after imputation

In [0]:
df[df.age.isna()].shape

In [0]:
df.age.plot(kind='hist', edgecolor='black')

In [0]:
sns.distplot(df.age)

### Convert objects to numeric

Most machine learning libraries in python require that the columns/features used are numeric. Let's see the data types of our data.

In [0]:
df.dtypes

We can see that `sex`, `embarked`, `title`are categorical values. 

Note: pandas assumes that string are also object.

pandas provides a function to convert categorical variable into dummy/indicator variables

In [0]:
pd.get_dummies(df).head() # by default pandas converts all string/categorical columns to dummy columns. So we should be careful when converting categorical columns to dummy columns.

In [0]:
df = pd.get_dummies(df, columns=['sex', 'embarked', 'title'], drop_first=True)
## if we have missing values in our dataset and instead of imputing it we can assign a new dummy level for the same. Example: title_NA
## drop_first = True drops the first level. This is sometimes required to maintain linear independence assumption in linear regression. 

### Saving python objects to pickle

Any object in python can be pickled so that it can be saved on disk. For example it can be a list, function, dataframe or a calibrated machine learning model. Let's save the dataframe we have created to a pickle file.

In [0]:
import pickle

with open("titanic_processed.pkl", "wb") as f:
  pickle.dump(df, f)

### Additional Resources

https://pandas.pydata.org/pandas-docs/stable/index.html